First, I'll import the libraries I may want. I probably won't use most of these for this first notebook, but I like to over-import just to be sure.

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

import requests
from bs4 import BeautifulSoup
import time

Defining Pushshift url:

In [6]:
url = 'https://api.pushshift.io/reddit/search/submission'

Writing a function for using the Pushshift API to pull from Reddit, including a sleep function to account for Reddit's refusal to let us be free:

In [92]:
def get_reddit(subreddit, num):
    
    params = {
    'subreddit' : subreddit,
    'size' : 100,
    }
    
    res = requests.get(url, params)
    data = res.json()
    posts = data['data']
    df = pd.DataFrame(posts)
    time.sleep(5)
    
    for i in range(num):
        params = {
        'subreddit' : subreddit,
        'size' : 100,
        'before' : df.iloc[-1]['created_utc']
        }
        res = requests.get(url, params)
        data = res.json()
        posts = data['data']
        new_df = pd.DataFrame(posts)
        df = df.append(new_df)
        time.sleep(5)
                    
        
    df = df[['subreddit', 'title', 'created_utc', 'selftext']]    
    df = df[df.selftext != '[removed]']
    df = df[df.selftext != '[deleted]']
    df = df[df.selftext != '']
    
    return df

In [88]:
nosleep = get_reddit('nosleep', 13)

In [91]:
nosleep.shape

(1045, 4)

In [98]:
# Calling creepypasta with a significantly higher iterate, because of a higher number of link/picture posts

creepy = get_reddit('creepypasta', 30)

In [99]:
creepy.shape

(1089, 4)

Creating a mondo dataframe containing both the nosleep and creepypasta data:

In [100]:
combo = pd.concat([nosleep, creepy])

In [101]:
combo.shape

(2134, 4)

Dropping the timestamp column, which is no longer necessary:

In [102]:
combo = combo.drop(columns = 'created_utc')

In [103]:
combo.shape

(2134, 3)

In [104]:
combo.isnull().sum()

subreddit    0
title        0
selftext     6
dtype: int64

In [105]:
combo = combo.dropna()

Dropping null values in selftext column, to make sure I only have useful information:

In [106]:
combo.shape

(2128, 3)

Saving combo as a csv so I can access it in other notebooks:

In [107]:
combo.to_csv('./Data/combo.csv', index = False)